In [5]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

🧠 Ejercicio propuesto
Título: Clasificador de reseñas de productos
Objetivo: Crear una pequeña app en Python usando LangChain que reciba reseñas de productos (texto libre), y las clasifique como positiva, negativa o neutral. Además, debe extraer el aspecto principal mencionado (por ejemplo: "batería", "pantalla", etc.).

🧩 Lo que pondrás en práctica
LLM: para generar respuestas naturales.
PromptTemplate: para estructurar la tarea.
Few-shot: para mejorar la comprensión del modelo.
Parsers: para interpretar la salida en JSON.
Chains: para orquestar todo el flujo.

💻 Código de ejemplo con solución
A continuación, te muestro un script completo en Python:

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
import json # Importar json para convertir dicts a JSON strings

# 1. Define ejemplos para few-shot con output como JSON string
examples = [
    {
        "input": "La batería dura muy poco, apenas un día.",
        "output": "negativo"
        
    },
    {
        "input": "Me encantó la pantalla, se ve muy nítida.",
        "output": "positivo"
        
    },
    {
        "input": "El diseño está bien, nada fuera de lo normal.",
        "output": "neutral"
        
    }
]

# 2. Define cómo se verán esos ejemplos en el prompt
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Reseña: {input}\nRespuesta: {output}"
)

# 3. Define el esquema de salida (Pydantic model) y el parser
class parsed_output(BaseModel):
    sentiment: str = Field(description="positivo, negativo o neutral")

output_parser = JsonOutputParser(pydantic_object=parsed_output)
raw_format_instructions = output_parser.get_format_instructions()

# Escapar las secuencias problemáticas en format_instructions.
# Langchain interpreta las cadenas literales como '{"foo": ...}' y '{"properties": ...}' 
# (que están dentro del texto de ejemplo de format_instructions) como placeholders para 
# variables llamadas '"foo"' y '"properties"' (es decir, los nombres de variable incluyen las comillas).
# Para que estas cadenas se traten como texto literal en el prompt final, 
# la plantilla que Langchain procesa debe contenerlas escapadas con llaves dobles.
# Por ejemplo, si queremos el literal '{"foo"}', la plantilla debe tener '{{{"foo"}}}'.
escaped_format_instructions = raw_format_instructions.replace('{"properties"}', '{{{"properties"}}}')
escaped_format_instructions = escaped_format_instructions.replace('{"foo"}', '{{{"foo"}}}')

# 4. Define el prompt completo con few-shot, incluyendo las instrucciones de formato en el prefix
# Las format_instructions se incluyen directamente en el prefix para guiar al LLM.
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=f"Clasifica la siguiente reseña de producto como positiva, negativa o neutral. Responde en JSON.\n{escaped_format_instructions}",
    suffix="Reseña: {input}\nRespuesta:",
    input_variables=["input"] # Solo 'input' se llenará en tiempo de ejecución para la reseña final
)

# 5. El prompt final es el few_shot_prompt ya configurado
final_prompt = few_shot_prompt

# 6. Crea el modelo LLM
# Asegúrate de tener configurada tu API key de OpenAI (OPENAI_API_KEY) en tu entorno.
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")  # Puedes usar "gpt-4" si tienes acceso

# 7. Define la chain completa usando LCEL (LangChain Expression Language)
# El orden correcto es: prompt -> llm -> parser
chain = final_prompt | llm | output_parser

# 8. Prueba el sistema
if __name__ == "__main__":
    test_input = "La cámara es buena, pero esperaba más por el precio."
    # Al invocar la cadena, la entrada debe ser un diccionario donde las claves
    # coincidan con las input_variables del prompt (en este caso, "input").
    result = chain.invoke({"input": test_input})
    print("Entrada de prueba:", test_input)
    print("Resultado parseado:", result)

    test_input_2 = "El rendimiento es increíble, todo fluye muy rápido."
    result_2 = chain.invoke({"input": test_input_2})
    print("\nEntrada de prueba:", test_input_2)
    print("Resultado parseado:", result_2)

KeyError: 'Input to FewShotPromptTemplate is missing variables {\'"properties"\', \'"foo"\'}.  Expected: [\'"foo"\', \'"properties"\', \'input\'] Received: [\'input\']\nNote: if you intended {"properties"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{"properties"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

🎯 Caso adaptado: Asistente de preparación para entrevistas de trabajo
Objetivo:
Crear una herramienta que:
Analice respuestas escritas a preguntas comunes de entrevistas.
Clasifique el desempeño como "bueno", "regular" o "mejorable".
Extraiga el aspecto clave mencionado (por ejemplo, "liderazgo", "trabajo en equipo", "resolución de problemas").
Sugerencia de mejora si el desempeño no fue "bueno".

💡 Lo que aplicarás
LLM para análisis semántico.
PromptTemplate con few-shot.
Output parser en JSON estructurado.
Chain ProgressBarara flujo modular.

In [ ]:
import json # Necesario para json.dumps
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
# from langchain.chains import LLMChain # No se usa directamente LLMChain con LCEL
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# 1. Ejemplos para few-shot
# Modificamos el 'output' para que sea una cadena JSON formateada como espera el parser,
# incluyendo los delimitadores ```json ... ```.
examples = [
    {
        "input": "Creo que soy una persona responsable y siempre trato de cumplir mis tareas. Me esfuerzo por aprender, aunque a veces me cuesta organizarme.",
        "output": f"```json\n{json.dumps({
            'evaluacion': 'regular',
            'aspecto_clave': 'organización',
            'sugerencia': 'Podrías mencionar ejemplos concretos donde hayas mejorado tu organización o cómo estás trabajando en ello.'
        }, indent=2)}\n```"
    },
    {
        "input": "En mi trabajo anterior lideré un equipo de 5 personas para lanzar una nueva funcionalidad en tiempo récord.",
        "output": f"```json\n{json.dumps({
            'evaluacion': 'bueno',
            'aspecto_clave': 'liderazgo',
            'sugerencia': None  # json.dumps convertirá None a null
        }, indent=2)}\n```"
    },
    {
        "input": "No tengo mucha experiencia, pero me gustaría aprender.",
        "output": f"```json\n{json.dumps({
            'evaluacion': 'mejorable',
            'aspecto_clave': 'experiencia',
            'sugerencia': 'Enfócate en habilidades transferibles o ejemplos de tu formación que demuestren potencial.'
        }, indent=2)}\n```"
    }
]

# 2. Prompt de ejemplo
# Este prompt ahora usará el 'output' que es una cadena JSON formateada.
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Respuesta: {input}\nEvaluación: {output}" # {output} será la cadena JSON con ```json ... ```
)

# 3. Few-shot prompt
# Modificamos el 'suffix' para incluir las 'format_instructions'.
# 'format_instructions' se proveerá mediante .partial() más adelante.
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Evalúa la respuesta del candidato como 'bueno', 'regular' o 'mejorable'. Extrae el aspecto clave mencionado y sugiere una mejora si no es buena. Devuelve solo JSON como se especifica a continuación.",
    suffix="Respuesta: {input}\n{format_instructions}\nEvaluación:", # Aquí se insertarán las instrucciones de formato
    input_variables=["input"], # 'format_instructions' es una variable parcial
    example_separator="\n\n" # Es buena práctica añadir un separador claro
)

# 4. Parser de salida
response_schemas = [
    ResponseSchema(name="evaluacion", description="bueno, regular o mejorable"),
    ResponseSchema(name="aspecto_clave", description="habilidad o aspecto mencionado (como liderazgo, experiencia, organización, etc.)"),
    ResponseSchema(name="sugerencia", description="consejo de mejora si la evaluación no fue buena (puede ser null o una cadena vacía si no aplica)")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 5. Prompt final con instrucciones de formato
# .partial() llenará la variable {format_instructions} en el suffix del few_shot_prompt
final_prompt_template = few_shot_prompt.partial(format_instructions=output_parser.get_format_instructions())

# 6. LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo") # Considera usar un modelo más reciente como "gpt-3.5-turbo-0125" o "gpt-4-turbo-preview" si es posible

# 7. Chain (usando Langchain Expression Language - LCEL)
# La cadena ya está correctamente definida con LCEL.
chain = final_prompt_template | llm | output_parser

# 8. Prueba
if __name__ == "__main__":
    test_input = "Me gusta colaborar con otros y suelo tomar la iniciativa cuando hay problemas. Sin embargo, a veces me cuesta delegar tareas."
    try:
        result = chain.invoke({"input": test_input}) # En LCEL, el input es un diccionario
        print("Resultado de la evaluación:")
        print(f"  Evaluación: {result.get('evaluacion')}")
        print(f"  Aspecto Clave: {result.get('aspecto_clave')}")
        print(f"  Sugerencia: {result.get('sugerencia')}")
    except Exception as e:
        print(f"Ocurrió un error durante la invocación: {e}")
        # Si hay un error de parsing, a menudo es útil imprimir la salida cruda del LLM
        # para depurar. Puedes hacerlo añadiendo un paso intermedio o modificando la cadena temporalmente.
        # raw_llm_output = (final_prompt_template | llm).invoke({"input": test_input})
        # print(f"Salida cruda del LLM: {raw_llm_output.content}")



KeyError: "'evaluacion'"